# 🧪 PyTorch Lab 2: Autoencoder 


## 0) Setup


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
print('PyTorch version:', torch.__version__)

PyTorch version: 2.8.0+cpu


## 1) Data: load Fashion-MNIST
**Exercise 1.1** — Load the training and test sets with `ToTensor()` transforms. Keep the default split.

Hints:
- Use `datasets.FashionMNIST` with `train=True/False`.
- Use a local folder like `data/` for `root`.
- Set `download=True` on first run.

In [ ]:
# Charger le dataset d'entraînement avec la transformation ToTensor()
train_data = datasets.FashionMNIST(
    root='data/',       # dossier local pour stocker les données
    train=True,         # jeu d'entraînement
    download=True,      # télécharge si non présent
    transform=ToTensor()  # transformation des images en tenseurs
)

# Charger le dataset de test de la même façon
test_data = datasets.FashionMNIST(
    root='data/',
    train=False,        # jeu de test
    download=True,
    transform=ToTensor()
)

print(f"Train set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

**Exercise 1.2** — Visualize one sample image to verify shapes and ranges.

In [ ]:
# Récupérer une image et son label depuis le dataset d'entraînement
image, label = train_data[0]  # premier échantillon

print(f"Label: {label}")
print(f"Image shape: {image.shape}")  # forme tensorielle, normalement [1, 28, 28]
print(f"Valeurs min/max pixel: {image.min().item()}/{image.max().item()}")  # valeurs normalisées

# Afficher l'image (tensor CxHxW, ici C=1)
plt.imshow(image.squeeze(), cmap='gray')  # retirer la dimension canal pour l'affichage
plt.title(f"Label: {label}")
plt.axis('off')
plt.show()

## 2) Dataloaders
**Exercise 2.1** — Create dataloaders for train and test with `batch_size=64`.


In [ ]:
# Création du DataLoader pour le jeu d'entraînement
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Création du DataLoader pour le jeu de test
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

print(f"Train loader batches: {len(train_loader)}")
print(f"Test loader batches: {len(test_loader)}")

## 3) Device
**Exercise 3.1** — Pick `cuda` if available, else `cpu`.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 4) Model: a tiny autoencoder
We’ll encode 28×28 images → 2D latent, then decode back to 28×28.

**Exercise 4.1** — Implement the following architecture:
- Encoder: Linear(784→256) → Tanh → Linear(256→10)

Return the output reshaped to the original image shape.


In [ ]:
# TODO: Exercise 4.1 — implement the autoencoder
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Aplatir l'entrée (images 28x28) en vecteur 784
        self.flatten = nn.Flatten()
        # Encoder sequence : 784 -> 256 -> tanh -> 10
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.Tanh(),
            nn.Linear(256, 10)
        )
        
    def forward(self, x):
        initial_shape = x.shape  # ex : (batch, 1, 28, 28)
        x = self.flatten(x)      # aplati en (batch, 784)
        x = self.encoder(x)      # sortie (batch, 10)
        # Ici, selon consigne, retourne la sortie telle quelle.
        # Si vous devez la reformer en image, il faut un decodeur (à préciser)
        return x

# Instanciation et transfert sur device (cpu ou gpu)
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): Tanh()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
)


## 5) Loss and Optimizer
**Exercise 5.1** — Use MSE loss to measure reconstruction error. Choose Adam with learning rate 1e-3.


In [ ]:
# Critère de perte : mean squared error pour erreur de reconstruction
loss_fn = nn.MSELoss()

# Optimiseur Adam pour les paramètres du modèle
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print(loss_fn)
print(optimizer)

## 6) Training & Evaluation loops
**Exercise 6.1** — Implement a standard training loop.



In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    for X_batch, _ in dataloader:  # on ne s'intéresse pas aux labels pour l'autoencodeur
        X_batch = X_batch.to(device)
        
        # Forward pass
        output = model(X_batch)
        
        # Calculer la perte entre sortie et entrée (reconstruction)
        loss = loss_fn(output, X_batch.view(output.shape))
        
        # Backward pass et optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * X_batch.size(0)
    
    avg_loss = total_loss / len(dataloader.dataset)
    print(f"Train loss: {avg_loss:.4f}")
    return avg_loss

**Exercise 6.2** — Implement a simple test loop computing loss and accuracy.

In [ ]:
def test_loop(dataloader, model, loss_fn, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, _ in dataloader:
            X_batch = X_batch.to(device)
            output = model(X_batch)
            loss = loss_fn(output, X_batch.view(output.shape))
            total_loss += loss.item() * X_batch.size(0)
    
    avg_loss = total_loss / len(dataloader.dataset)
    print(f"Test loss: {avg_loss:.4f}")
    return avg_loss


**Exercise 6.3** — Train for a few epochs (e.g., 5) and observe the printed losses.

In [ ]:
n_epochs = 5

for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")
    
    train_loss = train_loop(train_loader, model, loss_fn, optimizer, device)
    test_loss = test_loop(test_loader, model, loss_fn, device)
    
    print("-" * 30)
